# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

In [0]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_test.shape

(10000, 28, 28)

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [6]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [7]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
x_train_lt5 = x_train_lt5.reshape(x_train_lt5.shape[0], 28,28,1)
x_test_lt5 = x_test_lt5.reshape(x_test_lt5.shape[0], 28,28,1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')

In [0]:
x_train_lt5 = x_train_lt5 / 255

x_test_lt5 = x_test_lt5 / 255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [11]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


In [12]:
import numpy as np
np.unique(y_train_lt5)

array([0, 1, 2, 3, 4], dtype=uint8)

### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5 = tf.keras.utils.to_categorical(y_train_lt5,num_classes=5)
y_test_lt5 = tf.keras.utils.to_categorical(y_test_lt5,num_classes=5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [14]:
import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential

Using TensorFlow backend.


In [15]:
model = Sequential()

model.add(Convolution2D(32,3,3, input_shape=(28,28,1)))
model.add(Activation('relu'))
          
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
          
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Dropout(0.25))      
                    

W0707 06:11:27.339493 140605743953792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until
W0707 06:11:27.347053 140605743953792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 06:11:27.350790 140605743953792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/usr/local/lib/python3.6/dist-p

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
Total params: 9,568
Trainable params: 9,568
Non-trainable params: 0
_________________________________________________________________


### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
                              
model.add(Dense(5))
model.add(Activation('softmax'))
                                        
          


### Print the training and test accuracy for 5 epochs

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


W0707 06:11:27.510734 140605743953792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0707 06:11:27.544953 140605743953792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]


In [20]:
model.fit(x_train_lt5, y_train_lt5, batch_size=32, nb_epoch=5, 
              validation_data=(x_test_lt5, y_test_lt5), callbacks=callback_list)
    


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
W0707 06:11:27.691688 140605743953792 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 11s 361us/step - loss: 0.0632 - acc: 0.9799 - val_loss: 0.0168 - val_acc: 0.9946
Epoch 2/5
30596/30596 [==============================] - 9s 302us/step - loss: 0.0192 - acc: 0.9937 - val_loss: 0.0127 - val_acc: 0.9953
Epoch 3/5
30596/30596 [==============================] - 9s 300us/step - loss: 0.0122 - acc: 0.9960 - val_loss: 0.0201 - val_acc: 0.9938
Epoch 4/5
30596/30596 [==============================] - 9s 300us/step - loss: 0.0082 - acc: 0.9973 - val_loss: 0.0044 - val_acc: 0.9982
Epoch 5/5
30596/30596 [==============================] - 9s 302us/step - loss: 0.0059 - acc: 0.9978 - val_loss: 0.0101 - val_acc: 0.9967


In [21]:
model.save

<bound method Network.save of <keras.engine.sequential.Sequential object at 0x7fe109905438>>

### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [23]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_1
conv2d_1is not trainable

activation_1
activation_1is not trainable

conv2d_2
conv2d_2is not trainable

activation_2
activation_2is not trainable

max_pooling2d_1
max_pooling2d_1is not trainable

dropout_1
dropout_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

activation_3
activation_3is not trainable

dense_2
dense_2 is trainable

activation_4
activation_4is not trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [24]:
x_train_gt5.shape

(29404, 28, 28)

In [25]:
y_train_gt5.shape

(29404,)

In [0]:
x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0], 28,28,1)
x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0], 28,28,1)

In [0]:
x_train_gt5 = x_train_gt5.astype('float32')
x_test_gt5 = x_test_gt5.astype('float32')


In [0]:
x_train_gt5 = x_train_gt5 / 255

x_test_gt5 = x_test_gt5 / 255

In [29]:
np.unique(y_train_gt5)

array([0, 1, 2, 3, 4], dtype=uint8)

In [0]:
y_train_gt5 = tf.keras.utils.to_categorical(y_train_gt5,num_classes=5)
y_test_gt5 = tf.keras.utils.to_categorical(y_test_gt5,num_classes=5)

### Check

Verify the shapes with the given below.

In [39]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [40]:
model.fit(x_train_gt5, y_train_gt5, batch_size=32, nb_epoch=5, 
              validation_data=(x_test_gt5, y_test_gt5), callbacks=callback_list)
    

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
  576/29404 [..............................] - ETA: 9s - loss: 1.7051 - acc: 0.6406

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 9s 304us/step - loss: 0.1007 - acc: 0.9717 - val_loss: 0.0260 - val_acc: 0.9916
Epoch 2/5
29404/29404 [==============================] - 9s 302us/step - loss: 0.0237 - acc: 0.9920 - val_loss: 0.0252 - val_acc: 0.9932
Epoch 3/5
29404/29404 [==============================] - 9s 302us/step - loss: 0.0138 - acc: 0.9954 - val_loss: 0.0216 - val_acc: 0.9944
Epoch 4/5
29404/29404 [==============================] - 9s 303us/step - loss: 0.0107 - acc: 0.9961 - val_loss: 0.0232 - val_acc: 0.9949
Epoch 5/5
29404/29404 [==============================] - 9s 302us/step - loss: 0.0065 - acc: 0.9978 - val_loss: 0.0237 - val_acc: 0.9940


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [49]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [50]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [62]:
twenty_test.target

array([2, 2, 2, ..., 2, 2, 1])

In [51]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vect = TfidfVectorizer()

In [0]:
train_feature = (vect.fit_transform(twenty_train.data))
test_feature = (vect.transform(twenty_test.data))

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression

In [57]:
model = LogisticRegression()
model.fit(train_feature,twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
model.score(train_feature,twenty_train.target)

0.9827204253433761

In [64]:
prediction = model.predict(test_feature)
prediction

array([2, 2, 2, ..., 2, 2, 1])

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(prediction,twenty_test.target)

0.8868175765645806